In [33]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder

## Load and Merge data

In [34]:
df_tracks = pd.read_csv("../DataBase/raw/tracks.tsv", sep="\t")

df_tracks.rename(
    columns={
        "genre_title": "genre_top"
    },
    inplace=True,
)

print("Tracks Analysis:")
print("Distinct tracks:", df_tracks["track_id"].nunique())
print(df_tracks.shape)
print(df_tracks.isna().sum())

Tracks Analysis:
Distinct tracks: 99995
(99995, 14)
track_id                0
album_title           911
album_tracks            0
artist_latitude     59788
artist_longitude    59788
artist_name             0
duration                0
favorites               0
genre_top           55362
genres                  0
genres_all              0
interest                0
listens                 0
title                   1
dtype: int64


In [35]:
df_echonest = pd.read_csv("../DataBase/raw/echonest_features.tsv", sep="\t")

print("Echonest Analysis:")
print("Distinct tracks:", df_echonest["track_id"].nunique())
print(df_echonest.shape)
print(df_echonest.isna().sum())
print(df_echonest["track_id"].nunique())

Echonest Analysis:
Distinct tracks: 11440
(11440, 9)
track_id              0
acousticness          0
danceability         19
energy                0
instrumentalness      0
liveness              0
speechiness         150
tempo                 0
valence              21
dtype: int64
11440


In [36]:
df_spectral = pd.read_csv("../DataBase/raw/spectral_features.tsv", sep="\t")

print("Spectral Analysis:")
print("Distinct tracks:", df_spectral["track_id"].nunique())
print(df_spectral.shape)
print(df_spectral.isna().sum())

Spectral Analysis:
Distinct tracks: 99995
(99995, 22)
track_id                          0
spectral_bandwidth_kurtosis_01    0
spectral_bandwidth_max_01         0
spectral_bandwidth_mean_01        0
spectral_bandwidth_median_01      0
spectral_bandwidth_min_01         0
spectral_bandwidth_skew_01        0
spectral_bandwidth_std_01         0
spectral_centroid_kurtosis_01     0
spectral_centroid_max_01          0
spectral_centroid_mean_01         0
spectral_centroid_median_01       0
spectral_centroid_min_01          0
spectral_centroid_skew_01         0
spectral_centroid_std_01          0
spectral_rolloff_kurtosis_01      0
spectral_rolloff_max_01           0
spectral_rolloff_mean_01          0
spectral_rolloff_median_01        0
spectral_rolloff_min_01           0
spectral_rolloff_skew_01          0
spectral_rolloff_std_01           0
dtype: int64


In [37]:
df_genres = pd.read_csv("../DataBase/raw/genres.csv")

print("Genre Analysis:")
print(df_genres.shape)
print(df_genres.isna().sum())

Genre Analysis:
(164, 5)
genre_id            0
genre_color         0
genre_handle        0
genre_parent_id    13
genre_title         0
dtype: int64


In [38]:
df = pd.merge(
    left=df_tracks,
    right=df_spectral,
    on="track_id",
    how="left",
)
df = pd.merge(
    left=df,
    right=df_echonest,
    on="track_id",
    how="left",
)
print(df.shape)

(99995, 43)


## Feature Engineering

In [39]:
# Drop features related to song definition
COLUMNS_TO_DROP = [
    "track_id",
    "album_title",
    "album_tracks",
    "artist_latitude",
    "artist_longitude",
    # "artist_name",
    "title",
    # "interest",
    "favorites",
    "listens",
    # "duration",
    "genres",
    "genres_all",
]
df.drop(columns=COLUMNS_TO_DROP, axis=1, inplace=True)
print(df.shape)

(99995, 33)


In [40]:
label_encoder = LabelEncoder()
df["artist_name"] = label_encoder.fit_transform(df["artist_name"])

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99995 entries, 0 to 99994
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   artist_name                     99995 non-null  int64  
 1   duration                        99995 non-null  int64  
 2   genre_top                       44633 non-null  object 
 3   interest                        99995 non-null  int64  
 4   spectral_bandwidth_kurtosis_01  99995 non-null  float64
 5   spectral_bandwidth_max_01       99995 non-null  float64
 6   spectral_bandwidth_mean_01      99995 non-null  float64
 7   spectral_bandwidth_median_01    99995 non-null  float64
 8   spectral_bandwidth_min_01       99995 non-null  float64
 9   spectral_bandwidth_skew_01      99995 non-null  float64
 10  spectral_bandwidth_std_01       99995 non-null  float64
 11  spectral_centroid_kurtosis_01   99995 non-null  float64
 12  spectral_centroid_max_01        

In [42]:
GENRE_MAPPING = {
    "Energique": ["Electronic", "Experimental", "Pop", "Hip-Hop"],
    "Rock": ["Rock", "Blues", "Instrumental"],
    "Acoustic": ["Classical", "Jazz"],
    "Traditionnal": ["Old-Time / Historic", "Folk", "Country", "International"]
}

def coarse_genre_top(genre_top):
    for genre_top_coarse in GENRE_MAPPING:
        if genre_top in GENRE_MAPPING[genre_top_coarse]:
            return genre_top_coarse
    return None

df["coarse_genre_top"] = df["genre_top"].apply(coarse_genre_top)

## Save dataset

In [43]:
df.to_csv("../DataBase/processed/merged_data.csv", index=False)